In [2]:
import tensorflow as tf 
import numpy as np
from tensorflow import keras 
from tensorflow.keras.layers import * 
from tensorflow.keras.optimizers import *

In [46]:
def couple_convs(input_tensor, filter_size, kernel_size, stride = 1,
                 padding = "same",kernel_initializer='he_normal' , 
                 activation = "relu", batch_normalize = False):
    
    output = Conv2D(filters=filter_size, padding=padding, 
                    kernel_initializer=kernel_initializer,
                    kernel_size=kernel_size, strides=stride)(input_tensor) 
    
    if batch_normalize: 
        output = BatchNormalization()(output)
    
    output = Activation(activation)(output)
    
    output = Conv2D(filters=filter_size, padding=padding, 
                    kernel_initializer=kernel_initializer,
                    kernel_size=kernel_size, strides=stride)(output) 
    
    if batch_normalize: 
        output = BatchNormalization()(output)
    
    output = Activation(activation)(output)
    return output

In [53]:
input_tensor1 = Input((572,572,1))
filter_size = 64
#### cnotraction path ##############
######### increasing the what and decreasing the where ############
cv1= couple_convs(input_tensor=input_tensor1,
                  filter_size=filter_size, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool1 = MaxPool2D(pool_size=2, strides=2)(cv1)
drop1 = Dropout(0.2)(pool1)

cv2= couple_convs(input_tensor=drop1,
                  filter_size=filter_size*2, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool2 = MaxPool2D(pool_size=2, strides=2)(cv2)
drop2 = Dropout(0.2)(pool2)

cv3= couple_convs(input_tensor=drop2,
                  filter_size=filter_size*4, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool3 = MaxPool2D(pool_size=2, strides=2)(cv3)
drop3 = Dropout(0.2)(pool3)

cv4= couple_convs(input_tensor=drop3,
                  filter_size=filter_size*8, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool4 = MaxPool2D(pool_size=2, strides=2)(cv4)
drop4 = Dropout(0.2)(pool4)

cv5= couple_convs(input_tensor=drop4,
                  filter_size=filter_size*16, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 

In [54]:
drop4

<tf.Tensor 'dropout_29/cond/Merge:0' shape=(?, 32, 32, 512) dtype=float32>

In [61]:
############# expanding path ################## 
cropped_cv4 = Cropping2D(cropping=(4,4))(cv4)
pool5 = UpSampling2D(size =(2,2))(cv5) 
cv6 = Conv2D(filters=filter_size*8, kernel_size=2, activation ='relu', 
             padding = 'same', kernel_initializer = 'he_normal')(pool5)
merge6 = concatenate([cropped_cv4,cv6],axis=3)
cv6= couple_convs(input_tensor=merge6,
                  filter_size=filter_size*8, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool6 = UpSampling2D(size =(2,2))(cv6) 
drop6 = Dropout(0.2)(pool6)


cropped_cv3 = Cropping2D(cropping=(16,16))(cv3)

merge7 = concatenate([cropped_cv3,drop6],axis=3)
cv7= couple_convs(input_tensor=merge7,
                  filter_size=filter_size*8, 
                  padding = "valid",
                  kernel_size=(3,3),batch_normalize=True) 
pool7 = UpSampling2D(size =(2,2))(cv7) 
drop7 = Dropout(0.2)(pool7)

cropped_cv2 = Cropping2D(cropping=(40,40))(cv2)
merge8 = concatenate([cropped_cv2,drop7],axis=3)

In [60]:
cv6

<tf.Tensor 'activation_76/Relu:0' shape=(?, 52, 52, 512) dtype=float32>